# Assignment 2

Assignment 2 is an extension on lab 5, where we use neural sequence models for NER. Assignment 2 asks you to put a linear chain CRF on top.

Questions will be stated at the end of this notebook, and there will also be a follow up oral assessment.

A.I. usage is allowed but must be **throughly documented**. Do not write down any parts of A.I. solution if you do not understand them. You will **lose all marks for that part** if you do so. You will be regarded as **plagirism** if you do not understand your solution and you do not document A.I. usage.

Disscussion with classmates are allowed and encouraged.

Submit this notebook **with all the running output saved**.
Name the file p02s\<id\>.ipynb

# Deadline
**Thursday 7-Nov-2024 23:55 p.m.**


## **Fine-tuning BERT for named-entity recognition**

In the lab part (adapted from [Transformers-Tutorials](https://github.com/NielsRogge/Transformers-Tutorials)) of this notebook, we are going to use **BertForTokenClassification** which is included in the [Transformers library](https://github.com/huggingface/transformers) by HuggingFace. This model has BERT as its base architecture, with a token classification head on top, allowing it to make predictions at the token level, rather than the sequence level. Named entity recognition is typically treated as a token classification problem, so that's what we are going to use it for.

This tutorial uses the idea of **transfer learning**, i.e. first pretraining a large neural network in an unsupervised way, and then fine-tuning that neural network on a task of interest. In this case, BERT is a neural network pretrained on 2 tasks: masked language modeling and next sentence prediction. Now, we are going to fine-tune this network on a NER dataset. Fine-tuning is supervised learning, so this means we will need a labeled dataset.

If you want to know more about BERT, I suggest the following resources:
* the original [paper](https://arxiv.org/abs/1810.04805)
* Jay Allamar's [blog post](http://jalammar.github.io/illustrated-bert/) as well as his [tutorial](http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)
* Chris Mccormick's [Youtube channel](https://www.youtube.com/channel/UCoRX98PLOsaN8PtekB9kWrw)
* Abbishek Kumar Mishra's [Youtube channel](https://www.youtube.com/user/abhisheksvnit)

The following notebook largely follows the same structure as the tutorials by Abhishek Kumar Mishra. For his tutorials on the Transformers library, see his [Github repository](https://github.com/abhimishra91/transformers-tutorials).

NOTE: this notebook assumes basic knowledge about deep learning, BERT, and native PyTorch. If you want to learn more Python, deep learning and PyTorch, I highly recommend cs231n by Stanford University and the FastAI course by Jeremy Howard et al. Both are freely available on the web.  

Now, let's move on to the real stuff!

#### **Importing Python Libraries and preparing the environment**

This notebook assumes that you have the following libraries installed:
* pandas
* numpy
* sklearn
* pytorch
* transformers
* seqeval

As we are running this in Google Colab, the only libraries we need to additionally install are transformers and seqeval (GPU version):

In [58]:
!pip install transformers seqeval[gpu]

In [59]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

As deep learning can be accellerated a lot using a GPU instead of a CPU, make sure you can run this notebook in a GPU runtime (which Google Colab provides for free! - check "Runtime" - "Change runtime type" - and set the hardware accelerator to "GPU").

We can set the default device to GPU using the following code (if it prints "cuda", it means the GPU has been recognized):

In [60]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


#### **Downloading and preprocessing the data**
Named entity recognition (NER) uses a specific annotation scheme, which is defined (at least for European languages) at the *word* level. An annotation scheme that is widely used is called **[IOB-tagging](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)**, which stands for Inside-Outside-Beginning. Each tag indicates whether the corresponding word is *inside*, *outside* or at the *beginning* of a specific named entity. The reason this is used is because named entities usually comprise more than 1 word.

Let's have a look at an example. If you have a sentence like "Barack Obama was born in Hawaï", then the corresponding tags would be   [B-PERS, I-PERS, O, O, O, B-GEO]. B-PERS means that the word "Barack" is the beginning of a person, I-PERS means that the word "Obama" is inside a person, "O" means that the word "was" is outside a named entity, and so on. So one typically has as many tags as there are words in a sentence.

So if you want to train a deep learning model for NER, it requires that you have your data in this IOB format (or similar formats such as [BILOU](https://stackoverflow.com/questions/17116446/what-do-the-bilou-tags-mean-in-named-entity-recognition)). There exist many annotation tools which let you create these kind of annotations automatically (such as Spacy's [Prodigy](https://prodi.gy/), [Tagtog](https://docs.tagtog.net/) or [Doccano](https://github.com/doccano/doccano)). You can also use Spacy's [biluo_tags_from_offsets](https://spacy.io/api/goldparse#biluo_tags_from_offsets) function to convert annotations at the character level to IOB format.

Here, we will use a NER dataset from [Kaggle](https://www.kaggle.com/namanj27/ner-dataset) that is already in IOB format. One has to go to this web page, download the dataset, unzip it, and upload the csv file to this notebook. Let's print out the first few rows of this csv file:

In [61]:
import pandas as pd

# CSV文件的路径是Colab根目录下，直接使用文件名
path = '/content/ner_datasetreference.csv'  # 替换 'filename.csv' 为实际文件名

# 读取CSV文件并进行解码，尝试使用 'unicode_escape' 编码（你可以根据实际情况调整编码方式）
print("Path to dataset files:", path)

Path to dataset files: /content/ner_datasetreference.csv


In [62]:
data = pd.read_csv("/content/ner_datasetreference.csv", encoding='unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Let's check how many sentences and words (and corresponding tags) there are in this dataset:

In [63]:
data.count()

,0
Sentence #,47959
Word,1048565
POS,1048575
Tag,1048575


As we can see, there are approximately 48,000 sentences in the dataset, comprising more than 1 million words and tags (quite huge!). This corresponds to approximately 20 words per sentence.

Let's have a look at the different NER tags, and their frequency:

In [64]:
print("Number of tags: {}".format(len(data.Tag.unique())))
frequencies = data.Tag.value_counts()
frequencies

Number of tags: 17


,count
Tag,
O,887908
B-geo,37644
B-tim,20333
B-org,20143
I-per,17251
B-per,16990
I-org,16784
B-gpe,15870
I-geo,7414


There are 8 category tags, each with a "beginning" and "inside" variant, and the "outside" tag. It is not really clear what these tags mean - "geo" probably stands for geographical entity, "gpe" for geopolitical entity, and so on. They do not seem to correspond with what the publisher says on Kaggle. Some tags seem to be underrepresented. Let's print them by frequency (highest to lowest):

In [65]:
tags = {}
for tag, count in zip(frequencies.index, frequencies):
    if tag != "O":
        if tag[2:5] not in tags.keys():
            tags[tag[2:5]] = count
        else:
            tags[tag[2:5]] += count
    continue

print(sorted(tags.items(), key=lambda x: x[1], reverse=True))

[('geo', 45058), ('org', 36927), ('per', 34241), ('tim', 26861), ('gpe', 16068), ('art', 699), ('eve', 561), ('nat', 252)]


Let's remove "art", "eve" and "nat" named entities, as performance on them will probably be not comparable to the other named entities.

In [66]:
entities_to_remove = ["B-art", "I-art", "B-eve", "I-eve", "B-nat", "I-nat"]
data = data[~data.Tag.isin(entities_to_remove)]
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


We create 2 dictionaries: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook.

In [67]:
labels_to_ids = {k: v for v, k in enumerate(data.Tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(data.Tag.unique())}
labels_to_ids

{'O': 0,
 'B-geo': 1,
 'B-gpe': 2,
 'B-per': 3,
 'I-geo': 4,
 'B-org': 5,
 'I-org': 6,
 'B-tim': 7,
 'I-per': 8,
 'I-gpe': 9,
 'I-tim': 10}

As we can see, there are now only 10 different NER tags.

Now, we have to ask ourself the question: what is a training example in the case of NER, which is provided in a single forward pass? A training example is typically a **sentence**, with corresponding IOB tags. Let's group the words and corresponding tags by sentence:

In [68]:
# pandas has a very handy "forward fill" function to fill missing values based on the last upper non-nan value
data = data.fillna(method='ffill')
data.head()

<ipython-input-68-22549ca49efa>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [70]:
# let's create a new column called "sentence" which groups the words by sentence
data['sentence'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
# let's also create a new column called "word_labels" which groups the tags by sentence
data['word_labels'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
data.head()

,Sentence #,Word,POS,Tag,sentence,word_labels
0,Sentence: 1,Thousands,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
1,Sentence: 1,of,IN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
2,Sentence: 1,demonstrators,NNS,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
3,Sentence: 1,have,VBP,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
4,Sentence: 1,marched,VBN,O,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."


Let's only keep the "sentence" and "word_labels" columns, and drop duplicates:

In [71]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,Thousands of demonstrators have marched throug...,"O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B-..."
1,Families of soldiers killed in the conflict jo...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O,..."
2,They marched from the Houses of Parliament to ...,"O,O,O,O,O,O,O,O,O,O,O,B-geo,I-geo,O"
3,"Police put the number of marchers at 10,000 wh...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
4,The protest comes on the eve of the annual con...,"O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,B-org,I-org,O,..."


In [72]:
len(data)

47571

Let's verify that a random sentence and its corresponding tags are correct:

In [73]:
data.iloc[41].sentence

'Bedfordshire police said Tuesday that Omar Khayam was arrested in Bedford for breaching the conditions of his parole .'

In [74]:
data.iloc[41].word_labels

'B-gpe,O,O,B-tim,O,B-per,I-per,O,O,O,B-geo,O,O,O,O,O,O,O,O'

#### **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [75]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


A tricky part of NER with BERT is that BERT relies on **wordpiece tokenization**, rather than word tokenization. This means that we should also define the labels at the wordpiece-level, rather than the word-level!

For example, if you have word like "Washington" which is labeled as "b-gpe", but it gets tokenized to "Wash", "##ing", "##ton", then one approach could be to handle this by only train the model on the tag labels for the first word piece token of a word (i.e. only label "Wash" with "b-gpe"). This is what was done in the original BERT paper, see Github discussion [here](https://github.com/huggingface/transformers/issues/64#issuecomment-443703063).

Note that this is a **design decision**. You could also decide to propagate the original label of the word to all of its word pieces and let the model train on this. In that case, the model should be able to produce the correct labels for each individual wordpiece. This was done in [this NER tutorial with BERT](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L118). Another design decision could be to give the first wordpiece of each word the original word label, and then use the label “X” for all subsequent subwords of that word. All of them seem to lead to good performance.

Below, we define a regular PyTorch [dataset class](https://pytorch.org/docs/stable/data.html) (which transforms examples of a dataframe to PyTorch tensors). Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above. Word pieces that should be ignored have a label of -100 (which is the default `ignore_index` of PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)).

For more information about BERT's inputs, see [here](https://huggingface.co/transformers/glossary.html).








In [76]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.sentence[index].strip().split()

        word_labels = self.data.word_labels[index].split(",")

        #assert len(sentence) == len(word_labels) , it actually failed

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_split_into_words=True,
                             return_offsets_mapping=True,
                             padding='max_length',
                             truncation=True,
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels]
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        # pytorch loss ignores label with -100,
        # see https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        return item

  def __len__(self):
        return self.len

Now, based on the class we defined above, we can create 2 datasets, one for training and one for testing. Let's use a 80/20 split:

In [77]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (47571, 2)
TRAIN Dataset: (38057, 2)
TEST Dataset: (9514, 2)


Let's have a look at the first training example:

In [78]:
training_set[0]

{'input_ids': tensor([  101, 23564, 21030,  2099,  4967,  2001,  9388,  1011,  6109,  2005,
          2634,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

Let's verify that the input ids and corresponding targets are correct:

In [79]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
za          3
##hee       -100
##r         -100
khan        8
was         0
mar         0
-           -100
93          -100
for         0
india       1
.           0
[SEP]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[

Now, let's define the corresponding PyTorch dataloaders:

In [80]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#### **Defining the model**

Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased". The only thing we need to additionally specify is the number of labels (as this will determine the architecture of the classification head).

Note that only the base layers are initialized with the pretrained weights. The token classification head of top has just randomly initialized weights, which we will train, together with the pretrained weights, using our labelled dataset. This is also printed as a warning when you run the code cell below.

Then, we move the model to the GPU.

In [81]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

#### **Training the model**

Before training the model, let's perform a sanity check, which I learned thanks to Andrej Karpathy's wonderful [cs231n course](http://cs231n.stanford.edu/) at Stanford (see also his [blog post about debugging neural networks](http://karpathy.github.io/2019/04/25/recipe/)). The initial loss of your model should be close to -ln(1/number of classes) = -ln(1/17) = 2.83.

Why? Because we are using cross entropy loss. The cross entropy loss is defined as -ln(probability score of the model for the correct class). In the beginning, the weights are random, so the probability distribution for all of the classes for a given token will be uniform, meaning that the probability for the correct class will be near 1/17. The loss for a given token will thus be -ln(1/17). As PyTorch's [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) (which is used by `BertForTokenClassification`) uses *mean reduction* by default, it will compute the mean loss for each of the tokens in the sequence for which a label is provided.

Let's verify this:



In [82]:
inputs = training_set[4]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(2.3233, device='cuda:0', grad_fn=<NllLossBackward0>)

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [83]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 11])

Next, we define the optimizer. Here, we are just going to use Adam with a default learning rate. One can also decide to use more advanced ones such as AdamW (Adam with weight decay fix), which is [included](https://huggingface.co/transformers/main_classes/optimizer_schedules.html) in the Transformers repository, and a learning rate scheduler, but we are not going to do that here.

In [84]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function. It is partly based on [a really good repository about multilingual NER](https://github.com/chambliss/Multilingual_NER/blob/master/python/utils/main_utils.py#L344).

In [85]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1] #batchsize x sequence length x ntags
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 10==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 10 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

And let's train the model!

In [86]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 10 training steps: 2.384012222290039
Training loss per 10 training steps: 1.8547377911480991
Training loss per 10 training steps: 1.4104100891521998
Training loss per 10 training steps: 1.1866687286284663
Training loss per 10 training steps: 1.0554051602759011
Training loss per 10 training steps: 0.9596966189496657
Training loss per 10 training steps: 0.882047106985186
Training loss per 10 training steps: 0.8177068036207011
Training loss per 10 training steps: 0.76598479019271
Training loss per 10 training steps: 0.7224338129981533
Training loss per 10 training steps: 0.6860234347900541
Training loss per 10 training steps: 0.6551021686545363
Training loss per 10 training steps: 0.6274721886008239
Training loss per 10 training steps: 0.600299944973174
Training loss per 10 training steps: 0.5770105675179907
Training loss per 10 training steps: 0.5551543930508444
Training loss per 10 training steps: 0.5351531523169938
Training loss per 10 training steps

#### **Evaluating the model**

Now that we've trained our model, we can evaluate its performance on the held-out test set (which is 20% of the data). Note that here, no gradient updates are performed, the model just outputs its logits.

In [87]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)

            outpus = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outpus[0]
            eval_logits = outpus[1]
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.append(labels)
            eval_preds.append(predictions)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ [ids_to_labels[id.item()] for id  in labels] for labels in eval_labels]
    predictions = [[ids_to_labels[id.item()] for id in preds] for preds in  eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

As we can see below, performance is quite good! Accuracy on the test test is > 93%.

In [88]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.0993817076086998
Validation loss per 100 evaluation steps: 0.12426588566291451
Validation Loss: 0.12452645529836616
Validation Accuracy: 0.9632087839316719


However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library:

In [89]:
from seqeval.metrics import classification_report

print (labels)
print (predictions)
print(classification_report(labels, predictions))

[['O', 'O', 'O', 'O', 'B-geo', 'O', 'B-geo', 'I-geo', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'I-tim', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-geo', 'I-geo', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'B-per', 'I-per', 'I-per', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'O', 'O', 'B-per', 'I-per', 'O', 'B-gpe', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'B-

Performance already seems quite good, but note that we've only trained for 1 epoch. An optimal approach would be to perform evaluation on a validation set while training to improve generalization.

#### **Inference**

The fun part is when we can quickly test the model on new, unseen sentences.
Here, we use the prediction of the **first word piece of every word** (which is how the model was trained).

*In other words, the code below does not take into account when predictions of different word pieces that belong to the same word do not match.*

In [90]:
sentence = "@HuggingFace is a company based in New York, but is also has employees working in Paris"

inputs = tokenizer(sentence.split(),
                    is_split_into_words=True,
                    return_offsets_mapping=True,
                    padding='max_length',
                    truncation=True,
                    max_length=MAX_LEN,
                    return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, attention_mask=mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

print(tokens)
print(token_predictions)
prediction = []
for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
  #only predictions on first word pieces are important
  if mapping[0] == 0 and mapping[1] != 0:
    prediction.append(token_pred[1])
  else:
    continue
assert len(sentence.split()) == len(prediction)
print(sentence.split())
print(prediction)
for s,p in zip(sentence.split(),prediction):
    print (s,p)

['[CLS]', '@', 'hugging', '##face', 'is', 'a', 'company', 'based', 'in', 'new', 'york', ',', 'but', 'is', 'also', 'has', 'employees', 'working', 'in', 'paris', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[P

#### **Saving the model for future use**

Finally, let's save the vocabulary (.txt) file, model weights (.bin) and the model's configuration (.json) to a directory, so that both the tokenizer and model can be re-loaded using the `from_pretrained()` class method.


In [91]:
import os

directory = "/model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')
print('This lab part is completed')

All files saved
This lab part is completed


## Assignment 2

We have 4 questions with 100 Marks in total


1.   full labels for wordpieces (30 Marks)
2.   retrain with new labels, report result and BIO violations (20 Marks)
3.   add transition violation scores and report result (20 Marks)
4.   train transition scores and report result (30 Marks)



#Q1

1.   Recall that there is a design decision regarding how to convert labels at token level.

You need to propagate the original label of the word to all of its word pieces and let the model train on this. For beginning tags, we should have the first worldpiece having B tag, and the remaining having I tag.

For example, if you have word like "Washington" which is labeled as "b-gpe", but it gets tokenized to "Wash", "##ing", "##ton", then you should have "b-gpe", "i-gpe", "i-gpe"

Implement this version of label conversion by creating a new dataset class.

30 Marks

In [92]:
class NewDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len) -> None:
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index) -> dict:
        # Step 1: Get the sentence and word labels
        sentence = self.data.sentence[index].strip().split()
        word_labels = self.data.word_labels[index].split(',')

        # Step 2: Use tokenizer to encode sentence (includes padding/truncation up to max length)
        encoding = self.tokenizer(sentence,
                                  is_split_into_words=True,
                                  return_offsets_mapping=True,
                                  padding='max_length',
                                  truncation=True,
                                  max_length=self.max_len,
                                  return_tensors="pt")

        # Step 3: Create token labels for all word pieces of each tokenized word
        # We are going to assign labels to wordpieces (tokens)
        labels = [labels_to_ids[label] for label in word_labels]

        # Create an empty array of "O" labels (default value)
        encoded_labels = np.ones(len(encoding['offset_mapping'][0]), dtype=int) * labels_to_ids['O']

        # Step 4: Align word labels with tokenized wordpieces
        i = 0
        previous_mapping = None
        for idx, mapping in enumerate(encoding['offset_mapping'][0]):
            if mapping[0] == 0 and mapping[1] != 0:  # First wordpiece
                # Assign the first wordpiece with the "B-" label
                encoded_labels[idx] = labels[i]
                previous_mapping = mapping
                i += 1
            elif idx > 0 and previous_mapping[1] == mapping[0]:
                # Subsequent wordpieces get the "I-" label
                encoded_labels[idx] = labels_to_ids[ids_to_labels[labels[i - 1]].replace('B', 'I')]
                previous_mapping = mapping

        # Step 5: Convert everything to PyTorch tensors
        item = {key: val.squeeze() for key, val in encoding.items()}
        item['labels'] = torch.tensor(encoded_labels, dtype=torch.long)

        return item

    def __len__(self) -> int:
        return self.len

In [93]:
### Testing Code Here
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = NewDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = NewDataset(test_dataset, tokenizer, MAX_LEN)

print(training_set[0])

FULL Dataset: (47571, 2)
TRAIN Dataset: (38057, 2)
TEST Dataset: (9514, 2)
{'input_ids': tensor([  101, 23564, 21030,  2099,  4967,  2001,  9388,  1011,  6109,  2005,
         2634,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,    

In [94]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]['input_ids']), training_set[0]['labels']):
    print(f'{token:10}  {label}')

[CLS]       0
za          3
##hee       8
##r         8
khan        8
was         0
mar         0
-           0
93          0
for         0
india       1
.           0
[SEP]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD]       0
[PAD] 

In [95]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0,  # no additional child processes are used
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
               'shuffle': True,
               'num_workers': 0,  # no additional child processes are used
               }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

#Q2


1.   Train the model on the new labels, report the testing set performance with classification_report (10 Marks).   
2.   Gather the statistics of BIO rule violations. $\dfrac{\mathrm{\#Violations}}{\mathrm{\#Predicted Labels}}$ (10 Marks).

A violation happens when "I-tag" is not preceeded by "I-tag" or "B-tag".

20 Marks in total for Q2



In [50]:
'''
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification
from sklearn.metrics import accuracy_score
from seqeval.metrics import classification_report
import os

# Define constants
MAX_LEN = 128
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

# Load the tokenizer and model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Dataset class
class NERDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        sentence = self.data.sentence[index].strip().split()
        word_labels = self.data.word_labels[index].split(",")

        encoding = self.tokenizer(sentence,
                                  is_split_into_words=True,
                                  return_offsets_mapping=True,
                                  padding='max_length',
                                  truncation=True,
                                  max_length=self.max_len)

        labels = [labels_to_ids[label] for label in word_labels]
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] != 0:
                encoded_labels[idx] = labels[i]
                i += 1

        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        return item

    def __len__(self):
        return self.len
'''

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [96]:
def new_valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)

            outpus = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outpus[0]
            eval_logits = outpus[1]
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = mask.view(-1) != 0 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.append(labels)
            eval_preds.append(predictions)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ [ids_to_labels[id.item()] for id  in labels] for labels in eval_labels]
    predictions = [[ids_to_labels[id.item()] for id in preds] for preds in  eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [97]:
import torch
from sklearn.metrics import accuracy_score

def new_train(training_loader, model, optimizer, device, MAX_GRAD_NORM) -> None:
    tr_loss, tr_accuracy = 0, 0  # training loss and training accuracy
    nb_tr_examples, nb_tr_steps = 0, 0  # the number of training samples and training steps
    tr_preds, tr_labels = [], []  # stores prediction labels and real labels
    model.train()  # put model in training mode

    for idx, batch in enumerate(training_loader):
        # Get input data and move them to the target device
        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.long)

        # Forward propagation through the model (BERT + CRF)
        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)

        # Extract the loss and logits from the model's output
        loss = outputs[0]  # CRF-based loss
        logits = outputs[1]  # BERT logits

        # Update training loss
        tr_loss += loss.item()

        nb_tr_steps += 1  # update the training step count
        nb_tr_examples += labels.size(0)  # Update the number of training samples

        # Print training loss every 10 steps
        if idx % 10 == 0:
            loss_step = tr_loss / nb_tr_steps
            print(f'Training loss per 10 training steps: {loss_step}')

        # Compute training accuracy
        flattened_targets = labels.view(-1)  # Flatten the labels (batch_size * seq_len)
        active_logits = logits.view(-1, model.num_labels)  # Flatten the logits (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, dim=1)  # Predictions (batch_size * seq_len)

        # Only compute accuracy at active labels (excluding padding)
        active_accuracy = mask.view(-1) != 0  # Create a mask to exclude padding tokens
        labels = torch.masked_select(flattened_targets, active_accuracy)  # Only select valid labels
        predictions = torch.masked_select(flattened_predictions, active_accuracy)  # Only select valid predictions

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())  # Accuracy per step
        tr_accuracy += tmp_tr_accuracy  # Accumulate the accuracy

        # Gradient clipping (to avoid exploding gradients)
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=MAX_GRAD_NORM)

        # Backward pass: Compute gradients and update model parameters
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model parameters

    # Compute average loss and accuracy for the epoch
    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f'Training loss epoch: {epoch_loss}')
    print(f'Training accuracy epoch: {tr_accuracy}')

In [98]:
# Training the model
#optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 10 training steps: 0.103402279317379
Training loss per 10 training steps: 0.062007387592033905
Training loss per 10 training steps: 0.05160320709858622
Training loss per 10 training steps: 0.04617173062457192
Training loss per 10 training steps: 0.04347922057821983
Training loss per 10 training steps: 0.04166382277274833
Training loss per 10 training steps: 0.04094332603157544
Training loss per 10 training steps: 0.040463873486913424
Training loss per 10 training steps: 0.03950231451035282
Training loss per 10 training steps: 0.03866147857855309
Training loss per 10 training steps: 0.0384421702658776
Training loss per 10 training steps: 0.037792521788998765
Training loss per 10 training steps: 0.03741559741849249
Training loss per 10 training steps: 0.036918428904227626
Training loss per 10 training steps: 0.03676719596305637
Training loss per 10 training steps: 0.036490132699146964
Training loss per 10 training steps: 0.03603209737942826
Training lo

In [99]:
# Validate the model
labels, predictions = new_valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.030828990042209625
Validation loss per 100 evaluation steps: 0.028093450592074652
Validation Loss: 0.028442768086273
Validation Accuracy: 0.9594930939771823


In [100]:
# Report performance
report = classification_report(labels, predictions, output_dict=True)
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

         geo       0.79      0.90      0.84      7378
         gpe       0.90      0.92      0.91      3021
         org       0.60      0.56      0.58      3964
         per       0.72      0.76      0.74      3367
         tim       0.83      0.84      0.83      4070

   micro avg       0.77      0.81      0.79     21800
   macro avg       0.77      0.79      0.78     21800
weighted avg       0.76      0.81      0.78     21800



In [101]:
def bio_violations(predictions) -> tuple[int, int, int, int, int]:
    violations = 0
    violations_in_b = 0
    violations_in_i = 0
    predicted_labels = 0  # Count of total predicted labels
    word_token = []

    # Remove the 'O' label and treat each successive B or I label as a sublist
    for batch_sentence in predictions:
        temp_token = []
        for token in batch_sentence:
            predicted_labels += 1  # Count each label
            if token != 'O':
                temp_token.append(token)  # Add non-O tokens to the temporary list
            else:
                if len(temp_token) > 0:  # When an entity is completed
                    word_token.append(temp_token)  # Add the completed entity to word_token list
                    temp_token = []  # Reset temp_token for the next entity
        if len(temp_token) > 0:  # Add any remaining entities at the end
            word_token.append(temp_token)

    # Split out non-first B tags in all sublists
    for token in word_token:
        for i in range(1, len(token)):
            if token[i].startswith('B'):
                word_token.append(token[:i])
                word_token.append(token[i:])
                del token[:]
                break

    word_token = [token for token in word_token if token]  # Remove empty sublists
    word_token_b = [token for token in word_token if token[0].startswith('B')]  # Sublists starting with 'B'
    word_token_i = [token for token in word_token if token[0].startswith('I')]  # Sublists starting with 'I'

    # Check for violations within entities starting with B
    for token in word_token_b:
        base = token[0][2:]  # Base entity type (without 'B-')
        for i in range(1, len(token)):
            if token[i][2:] != base:  # If the entity type changes within the entity
                violations += 1
                violations_in_b += 1

    # Check for violations within entities starting with I
    for token in word_token_i:
        base = token[0][2:]  # Base entity type (without 'I-')
        for i in range(1, len(token)):
            if token[i][2:] != base:  # If the entity type changes within the entity
                violations += 1
                violations_in_i += 1

    # Add the count of 'I' tags that directly follow 'O' as additional violations
    violations += len(word_token_i)

    # Return the calculated values
    return violations, predicted_labels, violations_in_b, violations_in_i, len(word_token_i)

In [102]:
# Calculate BIO violations
violations, predicted_labels, violations_in_b, violations_in_i, i_tags_after_o = bio_violations(predictions)

# Calculate violation rate
violation_rate = (violations / predicted_labels) * 100 if predicted_labels > 0 else 0

# Print the BIO violations statistics and violation rate
print(f"Total Violations: {violations}")
print(f"Predicted Labels: {predicted_labels}")
print(f"Violation Rate: {violation_rate:.2f}%")

Total Violations: 1279
Predicted Labels: 252283
Violation Rate: 0.51%


#[pytorch-crf](https://pytorch-crf.readthedocs.io/en/stable/) Tutorial

Before we start Q3 and Q4, we need a brief tutorial on a automatically differentiable pytorch based linear-chain CRF package.

In [103]:
!pip install pytorch-crf

In [104]:
from torchcrf import CRF
crf_model = CRF(len(labels_to_ids),batch_first =True)
seq_length = 3  # maximum sequence length in a batch
batch_size = 2  # number of samples in the batch
emissions = torch.randn( batch_size, seq_length, len(labels_to_ids))
   # emissions should be tr_logits = outputs[1] #batchsize x sequence length x ntags
tags = torch.tensor([[0, 1, 2],[ 4, 3, 0]], dtype=torch.long)  # (seq_length, batch_size)
attention_mask = torch.tensor([[1, 1, 1],[ 1, 1, 0]], dtype=torch.bool)  # (seq_length, batch_size)

loss = crf_model(emissions, tags, mask=attention_mask)
predictions = crf_model.decode(emissions, mask=attention_mask)
print ("loss:",loss)
print ("predictions:",predictions)

loss: tensor(-12.9946, grad_fn=<SumBackward0>)
predictions: [[7, 8, 1], [10, 8]]


In [105]:
#should produce only 0 tags
crf_model.transitions.data[:,:] = -1e6
crf_model.transitions.data[0,0] = 0
loss = crf_model(emissions, tags, mask=attention_mask)
predictions = crf_model.decode(emissions, mask=attention_mask)
print ("loss:",loss)
print ("predictions:",predictions)

loss: tensor(-2999998., grad_fn=<SumBackward0>)
predictions: [[0, 0, 0], [0, 0]]


In [106]:
#should produce 0 tags, and ends with a 1
crf_model.transitions.data[:,:] = -1e6
crf_model.transitions.data[0,0] = 0
crf_model.transitions.data[0,1] = 1e6
loss = crf_model(emissions, tags, mask=attention_mask)
predictions = crf_model.decode(emissions, mask=attention_mask)
print ("loss:",loss)
print ("predictions:",predictions)

loss: tensor(-3000000.5000, grad_fn=<SumBackward0>)
predictions: [[0, 0, 1], [0, 1]]



#Q3
Use pytorch-crf to add transition scores to rule out violations.

This can be achieved via manully setting the crf_model.transitions.data, then rewrite valid method.

Then re-evaluate with the same trained Bert model as in Q2

1.   Add transition scores to rule out violations and re-evaluate (10 Marks)
2.   Recompute new BIO violations. You should find no violation (10 Marks)


20 Marks in total for Q3


In [107]:
# Step 1: Initialize CRF Model and Transition Matrix
from torchcrf import CRF

crf_model = CRF(len(labels_to_ids), batch_first=True).to(device)

# Set transition matrix with disallowed transitions set to very low values
crf_model.transitions.data[:, :] = -1e6

# Define allowed transition rules
for label_from, idx_from in labels_to_ids.items():
    for label_to, idx_to in labels_to_ids.items():
        if label_to == 'O' or (label_from == 'O' and label_to.startswith('B')):
            crf_model.transitions.data[idx_from, idx_to] = 0
        elif label_from.startswith('B') and label_to.startswith('I') and label_from[2:] == label_to[2:]:
            crf_model.transitions.data[idx_from, idx_to] = 0
        elif label_from.startswith('I') and label_to.startswith('I') and label_from[2:] == label_to[2:]:
            crf_model.transitions.data[idx_from, idx_to] = 0
        elif label_from.startswith('B') and label_to.startswith('B') and label_from[2:] != label_to[2:]:
            crf_model.transitions.data[idx_from, idx_to] = 0
        elif label_from.startswith('I') and label_to.startswith('B') and label_from[2:] != label_to[2:]:
            crf_model.transitions.data[idx_from, idx_to] = 0
print("\nInitialized CRF Transition Matrix:\n", crf_model.transitions)


Initialized CRF Transition Matrix:
 Parameter containing:
tensor([[       0.,        0.,        0.,        0., -1000000.,        0.,
         -1000000.,        0., -1000000., -1000000., -1000000.],
        [       0., -1000000.,        0.,        0.,        0.,        0.,
         -1000000.,        0., -1000000., -1000000., -1000000.],
        [       0.,        0., -1000000.,        0., -1000000.,        0.,
         -1000000.,        0., -1000000.,        0., -1000000.],
        [       0.,        0.,        0., -1000000., -1000000.,        0.,
         -1000000.,        0.,        0., -1000000., -1000000.],
        [       0., -1000000.,        0.,        0.,        0.,        0.,
         -1000000.,        0., -1000000., -1000000., -1000000.],
        [       0.,        0.,        0.,        0., -1000000., -1000000.,
                0.,        0., -1000000., -1000000., -1000000.],
        [       0.,        0.,        0.,        0., -1000000., -1000000.,
                0.,       

In [108]:
# Step 2: Redefine Validation Function to Evaluate with CRF
def crf_valid(model, crf_model, test_loader) -> tuple[list, list]:
    model.eval()
    crf_model.eval()

    total_loss, total_accuracy = 0, 0
    eval_steps, total_examples = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(test_loader):
            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.bool)
            labels = batch['labels'].to(device, dtype=torch.long)

            # Forward pass
            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            emissions = outputs[1]

            # Compute CRF loss and decode predictions
            loss = -crf_model(emissions, labels, mask=mask)
            predictions = crf_model.decode(emissions, mask=mask)

            total_loss += loss.item()
            eval_steps += 1
            total_examples += labels.size(0)

            if idx % 10 == 0:
                print(f'Validation loss at step {idx + 1}: {total_loss / eval_steps:.4f}')

            # Flatten labels and predictions for accuracy calculation
            flattened_targets = labels.view(-1)
            active_accuracy = mask.view(-1)
            valid_labels = torch.masked_select(flattened_targets, active_accuracy).tolist()
            valid_predictions = [label for sentence in predictions for label in sentence]

            all_labels.append(valid_labels)
            all_preds.append(valid_predictions)

            step_accuracy = accuracy_score(valid_labels, valid_predictions)
            total_accuracy += step_accuracy

    labels = [[ids_to_labels[id] for id in label] for label in all_labels]
    predictions = [[ids_to_labels[id] for id in pred] for pred in all_preds]

    avg_loss = total_loss / eval_steps
    avg_accuracy = total_accuracy / eval_steps
    print(f'\nValidation Loss: {avg_loss:.4f}')
    print(f'Validation Accuracy: {avg_accuracy:.4f}')

    return labels, predictions

In [109]:
#Set CRF model and evaluate
# Re-evaluate the model after setting transition scores
labels, predictions = crf_valid(model, crf_model, testing_loader)

Validation loss at step 1: 150.9380
Validation loss at step 11: 164.2874
Validation loss at step 21: 185.2179
Validation loss at step 31: 191.2928
Validation loss at step 41: 190.7664
Validation loss at step 51: 189.7685
Validation loss at step 61: 193.3666
Validation loss at step 71: 189.8946
Validation loss at step 81: 188.9894
Validation loss at step 91: 11178.4067
Validation loss at step 101: 10087.1912
Validation loss at step 111: 9197.2323
Validation loss at step 121: 8452.5656
Validation loss at step 131: 7820.5556
Validation loss at step 141: 7278.5507

Validation Loss: 6898.0261
Validation Accuracy: 0.9600


In [110]:
print("\nClassification Report:")
print(classification_report(labels, predictions, zero_division=0))


Classification Report:
              precision    recall  f1-score   support

         geo       0.81      0.90      0.85      7378
         gpe       0.92      0.92      0.92      3021
         org       0.68      0.57      0.62      3964
         per       0.77      0.77      0.77      3367
         tim       0.86      0.84      0.85      4070

   micro avg       0.81      0.81      0.81     21800
   macro avg       0.81      0.80      0.80     21800
weighted avg       0.81      0.81      0.81     21800



In [112]:
# Step 4: Calculate and Print BIO Violations Statistics
violations, total_labels, violations_b, violations_i, start_i = bio_violations(predictions)

print("\nBIO Violations Statistics:")
print(f"Total Violations: {violations}")
print(f"Violation Rate: {violations / total_labels:.2%}")



BIO Violations Statistics:
Total Violations: 0
Violation Rate: 0.00%


#Q4
Use pytorch-crf to jointly re-train the Bert model with transition score.

1.   Train transition scores jointly with Bert model (20 Marks)
2.   Report Result and Recompute new BIO violations.  (10 Marks)


30 Marks in total for Q4



In [61]:
#optimizer = torch.optim.Adam(list(model.parameters()) + list(crf_model.parameters()), lr=LEARNING_RATE)

"\n# The parameters of BERT and CRF models are collected separately\noptimizer_parameters = [\n    {'params': model.parameters(), 'lr': LEARNING_RATE},  # BERT model parameters\n    {'params': crf_model.parameters(), 'lr': LEARNING_RATE}  # # CRF model parameters\n]\n\noptimizer = torch.optim.Adam(optimizer_parameters)\n"

In [113]:
# Step 2: Rewrite the training function for BERT + CRF
def train_crf_model(train_loader):
    model.train()
    crf_model.train()
    total_loss, total_accuracy = 0, 0
    total_steps, total_examples = 0, 0
    all_preds, all_labels = [], []

    for step, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.bool)
        labels = batch['labels'].to(device, dtype=torch.long)

        # Forward pass through BERT and CRF
        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        emissions = outputs[1]  # Get emissions for CRF layer
        loss = -crf_model(emissions, labels, mask=mask)  # Negative log likelihood for CRF
        predictions = crf_model.decode(emissions, mask=mask)  # Decoding for predictions

        # Accumulate loss
        total_loss += loss.item()
        total_steps += 1
        total_examples += labels.size(0)

        if step % 10 == 0:
            print(f'Training loss after {step + 1} steps: {total_loss / total_steps}')

        # Flatten labels and predictions for accuracy calculation
        flattened_labels = labels.view(-1)
        active_mask = mask.view(-1)
        valid_labels = torch.masked_select(flattened_labels, active_mask).tolist()
        valid_predictions = [label for sentence in predictions for label in sentence]

        all_labels.append(valid_labels)
        all_preds.append(valid_predictions)
        accuracy_step = accuracy_score(valid_labels, valid_predictions)
        total_accuracy += accuracy_step

        # Backpropagation with gradient clipping
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(list(model.parameters()) + list(crf_model.parameters()), max_norm=MAX_GRAD_NORM)
        optimizer.step()

    # Print epoch loss and accuracy
    avg_loss = total_loss / total_steps
    avg_accuracy = total_accuracy / total_steps
    print(f'Epoch Training Loss: {avg_loss}')
    print(f'Epoch Training Accuracy: {avg_accuracy}')


In [114]:
for epoch in range(EPOCHS):
    print(f"\nTraining Epoch {epoch + 1}/{EPOCHS}")
    train_crf_model(training_loader)


Training Epoch 1/1
Training loss after 1 steps: 198.82571411132812
Training loss after 11 steps: 170.33266865123403
Training loss after 21 steps: 156.25211479550316
Training loss after 31 steps: 154.94840732697517
Training loss after 41 steps: 150.12154034870426
Training loss after 51 steps: 144.05769527659697
Training loss after 61 steps: 142.8533893022381
Training loss after 71 steps: 141.91285426180127
Training loss after 81 steps: 137.5806551332827
Training loss after 91 steps: 135.44234005435482
Training loss after 101 steps: 134.11088433596169
Training loss after 111 steps: 132.3261451377525
Training loss after 121 steps: 130.64919142289594
Training loss after 131 steps: 129.20089817775116
Training loss after 141 steps: 127.91019734592302
Training loss after 151 steps: 126.6194827603978
Training loss after 161 steps: 126.07031252369377
Training loss after 171 steps: 125.75059377659134
Training loss after 181 steps: 125.02228135166906
Training loss after 191 steps: 123.7760399823

In [115]:
print("\nCRF Transition Matrix:")
print(crf_model.transitions)


CRF Transition Matrix:
Parameter containing:
tensor([[       0.,        0.,        0.,        0., -1000000.,        0.,
         -1000000.,        0., -1000000., -1000000., -1000000.],
        [       0., -1000000.,        0.,        0.,        0.,        0.,
         -1000000.,        0., -1000000., -1000000., -1000000.],
        [       0.,        0., -1000000.,        0., -1000000.,        0.,
         -1000000.,        0., -1000000.,        0., -1000000.],
        [       0.,        0.,        0., -1000000., -1000000.,        0.,
         -1000000.,        0.,        0., -1000000., -1000000.],
        [       0., -1000000.,        0.,        0.,        0.,        0.,
         -1000000.,        0., -1000000., -1000000., -1000000.],
        [       0.,        0.,        0.,        0., -1000000., -1000000.,
                0.,        0., -1000000., -1000000., -1000000.],
        [       0.,        0.,        0.,        0., -1000000., -1000000.,
                0.,        0., -1000000

In [116]:
labels, predictions = crf_valid(model, crf_model, testing_loader)
print("\nClassification Report:")
print(classification_report(labels, predictions, zero_division=0))

Validation loss at step 1: 109.5133
Validation loss at step 11: 98.8174
Validation loss at step 21: 92.3461
Validation loss at step 31: 90.4740
Validation loss at step 41: 91.8847
Validation loss at step 51: 93.1126
Validation loss at step 61: 94.0829
Validation loss at step 71: 94.8147
Validation loss at step 81: 94.5840
Validation loss at step 91: 11083.9241
Validation loss at step 101: 9996.0601
Validation loss at step 111: 9103.1804
Validation loss at step 121: 8359.2397
Validation loss at step 131: 7728.5642
Validation loss at step 141: 7187.7900

Validation Loss: 6806.7514
Validation Accuracy: 0.9640

Classification Report:
              precision    recall  f1-score   support

         geo       0.84      0.90      0.87      7378
         gpe       0.93      0.94      0.94      3021
         org       0.75      0.63      0.68      3964
         per       0.78      0.80      0.79      3367
         tim       0.90      0.86      0.88      4070

   micro avg       0.84      0.83   

In [117]:
violations, predicted_labels, violations_in_b, violations_in_i, start_i = bio_violations(predictions)
# Print BIO violations statistics
print("\nBIO Violations Statistics:")
print(f"Total Violations: {violations}")
print(f"Violation Rate: {violations / predicted_labels:.2%}")


BIO Violations Statistics:
Total Violations: 0
Violation Rate: 0.00%


# Appendix
All the prompts and AI responses:

## AI-Prompts
Prompt:

gpt4:


Prompt:

gpt4:

##Other References

本人遇到的经典报错：
1.不能使用GPU运行，该报错常见于运行单元不够用
解决方法：买GPU资源，也可以在代码中加入强制使用GPU，但一开始就是单元跑完了，没发现。
2.张量问题：
解决方法：告诉gpt，让gpt解决。应该是需要张量展平
3.Q2遇到的key value：-100问题
解决方法1：gpt解决
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0,  # no additional child processes are used
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
               'shuffle': True,
               'num_workers': 0,  # no additional child processes are used
               }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
加上这个
2.如果代码modify次数太多，需要重启会话
3.违规率和准确率过低问题（Q2）
问题原因：Q1的分类不准确，需要先checkQ1代码（问同学）
更重要的一个原因：在解决-100的bug的时候，gpt给的方案丢数据了
解决方法：清除gpt记忆，重新训练后跑通
4.数据集跑不过去：
解决方法：gpt也没有解决，询问同学后解决
原因：数据集的某一行有不能识别的标识，删掉就好
